In [18]:
import os
import json
import csv

# 입력 파일 경로
rating_csv = "result/image/image_rating_pairs.csv"
output_csv = "result/prompt/region_caption.csv"
os.makedirs(os.path.dirname(output_csv), exist_ok=True)

rows = []
with open(rating_csv, "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        # 이미지 경로에서 prompt.json 경로 추출
        image_path = row["image_path_raw"]
        prompt_path = image_path.replace("image.png", "prompt.json")
        region_caption = ""
        if os.path.isfile(prompt_path):
            with open(prompt_path, "r", encoding="utf-8") as pf:
                data = json.load(pf)
                region_caption = data.get("region_captions", "")
        rows.append([
            row["person"],
            row["system_type"],
            row["rating"],
            region_caption
        ])

# 결과 저장
with open(output_csv, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["participant", "system", "rating", "region_captions"])
    writer.writerows(rows)

print(f"Saved region_caption with participant, system, rating to {output_csv}")

Saved region_caption with participant, system, rating to result/prompt/region_caption.csv


In [19]:
import pandas as pd

# region_caption 데이터 불러오기
df = pd.read_csv("result/prompt/region_caption.csv")
df["rating"] = df["rating"].astype(float)

# participant_info.csv 불러오기
info = pd.read_csv("participant_info.csv")

# participant, system별 target 정보 매핑
target_map = {}
for _, row in info.iterrows():
    participant = row["participants"]
    target_map[(participant, "system1")] = row["System 1"]
    target_map[(participant, "system2")] = row["System 2"]

# system, participant별 rating이 가장 높은 row만 남김
df_max = df.loc[df.groupby(["system", "participant"])["rating"].idxmax()]
df_max["target"] = df_max.apply(lambda x: target_map.get((x["participant"], x["system"]), ""), axis=1)
# system-target별로 정렬
df_max.sort_values(by=["system", "target", "participant"], inplace=True)
# 컬럼 순서 변경
df_max = df_max[["system", "target", "region_captions", "participant", "rating"]]
df_max.to_csv("result/prompt/region_caption_max_rating.csv", index=False)
df_max

,system,target,region_captions,participant,rating
15,system1,target1,['an old man with white hair wearing a black b...,P11,3.0
20,system1,target1,"['a man wearing a black hat, staring at a woma...",P3,4.0
28,system1,target1,['a middle-aged woman holding a basket is walk...,P4,5.0
44,system1,target1,['an old man wearing a black beret and a gray ...,P5,7.0
64,system1,target1,"['a grandfather wearing a black hat, a white s...",P8,4.0
66,system1,target1,"['a grandfather with white hair, wearing a bla...",P9,5.0
0,system1,target2,"['a man wearing a white shirt, beige pants, an...",P1,6.0
72,system1,target2,"['a man standing holding a white paper bag, we...",P10,6.0
18,system1,target2,"['a male with short brown hair, around 180 cm,...",P12,3.0
81,system1,target2,"['a man wearing a white shirt and beige pants,...",P2,5.0


In [20]:
import ast, re, pandas as pd
from collections import Counter, defaultdict
from itertools import combinations
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
import numpy as np

# 0) 로드
df = pd.read_csv("result/prompt/region_caption_max_rating.csv")  # columns: system,target,region_captions,participant,rating

# 1) 전처리: 문자열 리스트를 진짜 리스트로
def parse_caps(x):
    try: return ast.literal_eval(x)
    except: return [x]
df["caps"] = df["region_captions"].apply(parse_caps)

# 2) 간단 사전/패턴
COLORS = {"black","white","grey","gray","blue","pink","yellow","beige","brown","green","navy","ivory","magenta"}
OBJECTS = {"man","woman","girl","grandfather","grandpa","father","mother","boy","car","basket","tray","bottles","balloon","teddy","bear","camera","map","sunglasses","luggage","hat","dress","cardigan","beret","shorts","sandals","shoes","pants","skirt","trees","building","city"}
REL_PATTERNS = [
    ("holding", r"\bholding\b.*\b(?P<obj>\w+)\b"),
    ("wearing", r"\bwearing\b.*\b(?P<obj>\w+)\b"),
    ("looking at", r"\blooking (at|to|toward)\b"),
    ("beside", r"\bbeside\b|\bnext to\b"),
    ("with", r"\bwith\b"),
    ("carrying", r"\bcarrying\b"),
    ("taking picture", r"\btaking a picture\b|\bholding a camera\b"),
]

KEY_CONSTRAINTS = {
  "yellow balloon": r"\byellow\b.*\bballoon\b",
  "teddy bear": r"\bteddy\b|\bbear\b",
  "flower-pattern dress": r"\bflower[- ]?pattern(ed)?\b.*\b(dress|one-?piece)\b",
  "water bottles on tray": r"\b(bottles?).*\b(tray|plate|pan)\b|\btray\b.*\bbottles?\b",
  "basket": r"\bbasket\b",
  "beret": r"\bberet\b",
  "map": r"\bmap\b",
  "camera": r"\bcamera\b|\btaking a picture\b",
  "sunglasses": r"\bsunglasses\b",
  "luggage": r"\bluggage\b|\b(suitcase|carrier|trunk)\b"
}

# 3) 파싱 유틸
def tokenize(s): return re.findall(r"[a-z]+", s.lower())
def extract_objects(tokens): return {t for t in tokens if t in OBJECTS}
def extract_colors(tokens): return {t for t in tokens if t in COLORS}
def adjective_ratio(text):
    # 규칙기반 대체: 형용사 후보(색상/길이/나이/패턴 키워드 등)로 근사
    ADJ_HINTS = COLORS | {"old","young","little","small","long","short","curly","wavy","brown","black","white","grey","dark","light","blue","pink","beige","navy","ivory","magenta","flower","pattern"}
    toks = tokenize(text)
    return sum(t in ADJ_HINTS for t in toks) / max(1, len(toks))

def constraint_hits(text):
    return {k: bool(re.search(p, text.lower())) for k,p in KEY_CONSTRAINTS.items()}

def relations(text):
    out = []
    for name, pat in REL_PATTERNS:
        if re.search(pat, text.lower()):
            out.append(name)
    return set(out)

# 4) 타깃별 기대치(골드) 만들기 (없다면 다수결 추정)
gold = defaultdict(lambda: {"objects":Counter(), "relations":Counter(), "constraints":Counter()})
for _,row in df.iterrows():
    for cap in row["caps"]:
        toks = tokenize(cap)
        gold[row["target"]]["objects"].update(extract_objects(toks))
        gold[row["target"]]["relations"].update(relations(cap))
        gold[row["target"]]["constraints"].update({k for k,v in constraint_hits(cap).items() if v})

def majority_set(counter, thresh=0.3):
    if not counter: return set()
    n = sum(counter.values())
    return {k for k,c in counter.items() if c/n >= thresh}

gold_sets = {}
for t in df["target"].unique():
    gold_sets[t] = {
        "objects": majority_set(gold[t]["objects"]),
        "relations": majority_set(gold[t]["relations"]),
        "constraints": majority_set(gold[t]["constraints"])
    }

# 5) 캡션 단위 → 참가자/시스템/타깃 단위 집계
rows = []
for _,r in df.iterrows():
    t = r["target"]; sys = r["system"]; pid = r["participant"]; rating = r["rating"]
    caps = r["caps"]
    all_text = " ".join(caps).lower()
    toks = tokenize(all_text)

    objs = extract_objects(toks)
    rels = set().union(*[relations(c) for c in caps])

    cons = constraint_hits(all_text)
    cons_rate = np.mean(list(cons.values())) if cons else 0.0

    adj_ratio = np.mean([adjective_ratio(c) for c in caps]) if caps else 0.0
    color_rate = np.mean([len(extract_colors(tokenize(c)))>0 for c in caps]) if caps else 0.0

    g = gold_sets[t]
    objcov = len(objs & g["objects"]) / max(1, len(g["objects"])) if g["objects"] else 0
    relcov = len(rels & g["relations"]) / max(1, len(g["relations"])) if g["relations"] else 0
    conscov = np.mean([cons.get(k,False) for k in g["constraints"]]) if g["constraints"] else cons_rate

    rows.append({
        "system": sys, "target": t, "participant": pid, "rating": rating,
        "obj_coverage": objcov, "rel_coverage": relcov, "constraint_coverage": conscov,
        "adj_ratio": adj_ratio, "color_presence_rate": color_rate
    })

metrics = pd.DataFrame(rows)

# 6) 시스템 요약 (타깃별 평균 후 전체 평균)
summary = (metrics
           .groupby(["system","target"])
           [["obj_coverage","rel_coverage","constraint_coverage","adj_ratio","color_presence_rate","rating"]]
           .mean()
           .groupby("system").mean()
           .sort_values("rating", ascending=False))

print(summary.round(3))
metrics

         obj_coverage  rel_coverage  constraint_coverage  adj_ratio  \
system                                                                
system2           0.0           1.0                0.400      0.143   
system1           0.0           1.0                0.425      0.144   

         color_presence_rate  rating  
system                                
system2                0.793   5.417  
system1                0.946   4.833  


,system,target,participant,rating,obj_coverage,rel_coverage,constraint_coverage,adj_ratio,color_presence_rate
0,system1,target1,P11,3.0,0,1.0,0.5,0.172766,1.000000
1,system1,target1,P3,4.0,0,1.0,0.4,0.089018,0.750000
2,system1,target1,P4,5.0,0,1.0,0.6,0.060944,0.600000
3,system1,target1,P5,7.0,0,1.0,0.5,0.199351,1.000000
4,system1,target1,P8,4.0,0,1.0,0.3,0.221400,1.000000
5,system1,target1,P9,5.0,0,1.0,0.4,0.201486,1.000000
6,system1,target2,P1,6.0,0,1.0,0.4,0.171116,1.000000
7,system1,target2,P10,6.0,0,1.0,0.4,0.129019,1.000000
8,system1,target2,P12,3.0,0,1.0,0.4,0.154530,1.000000
9,system1,target2,P2,5.0,0,1.0,0.4,0.101079,1.000000


In [21]:
import ast
region_captions_list = df_max['region_captions'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else [x])
df_caps = df_max[['system', 'target', 'participant']].copy()
df_caps['region_captions'] = region_captions_list
df_caps = df_caps.reset_index(drop=True)
df_caps

,target,system,region_captions,participant
0,target1,system1,[an old man with white hair wearing a black be...,P11
1,target1,system1,"[a man wearing a black hat, staring at a woman...",P3
2,target1,system1,[a middle-aged woman holding a basket is walki...,P4
3,target1,system1,[an old man wearing a black beret and a gray c...,P5
4,target1,system1,"[a grandfather wearing a black hat, a white sh...",P8
5,target1,system1,"[a grandfather with white hair, wearing a blac...",P9
6,target1,system2,"[an old white man wearing a grey cardigan, a w...",P7
7,target1,system2,"[an old guy wearing a black hat walking, a you...",P6
8,target1,system2,[a grandpa with white hair wearing a black cap...,P1
9,target1,system2,"[a grandfather wearing a black beret, a white ...",P10


In [22]:
# target1만 추출
df_target1 = df_caps[df_caps['target'] == 'target1'].copy()

df_target1_sys1 = df_target1[df_target1['system'] == 'system1'].copy()

# 각 region_caption의 길이
lengths = df_target1_sys1['region_captions'].apply(len)
# 각 region_caption의 첫 번째 값만 추출
first_captions = df_target1_sys1['region_captions'].apply(lambda x: x[0] if len(x) > 0 else None)
print('region_caption lengths:', lengths.tolist())
print('region_caption[0] list:')
for cap in first_captions:
    print(cap)

region_caption lengths: [4, 4, 5, 4, 6, 4]
region_caption[0] list:
an old man with white hair wearing a black beret, a grey cardigan with two black buttons, dark blue pants, and black shoes, looking left and a bit short, wearing a white shirt beneath the cardigan
a man wearing a black hat, staring at a woman and a man on his right
a middle-aged woman holding a basket is walking with her husband, who has a happy smile, and is wearing a flower-patterned one-piece
an old man wearing a black beret and a gray cardigan over a white shirt, also wearing navy pants and black shoes
a grandfather wearing a black hat, a white shirt, a gray cardigan, black suit pants, and formal shoes. He is white-haired and is looking at his granddaughter with a smile on his face.
a grandfather with white hair, wearing a black beret hat, a grey cardigan, navy pants, and black shoes, looking to the right side


In [23]:
df_target1_sys2 = df_target1[df_target1['system'] == 'system2'].copy()

# 각 region_caption의 길이
lengths = df_target1_sys2['region_captions'].apply(len)
# 각 region_caption의 첫 번째 값만 추출
first_captions = df_target1_sys2['region_captions'].apply(lambda x: x[0] if len(x) > 0 else None)
print('region_caption lengths:', lengths.tolist())
print('region_caption[0] list:')
for cap in first_captions:
    print(cap)

region_caption lengths: [8, 10, 9, 4, 8, 9]
region_caption[0] list:
an old white man wearing a grey cardigan, a white shirt, black pants, and a black artist-style hat, holding nothing related to a woman while looking at a girl
an old guy wearing a black hat walking
a grandpa with white hair wearing a black cap, a white shirt, a grey cardigan, and black long pants, and a pair of black shoes, watching a girl
a grandfather wearing a black beret, a white shirt, a gray cardigan, black pants, and black shoes, watching the right side
an old man wearing a gray cardigan with 2 buttons on the shirt, black pants, and a beret, with short white hair, around 150 cm tall, looking at a 35-year-old woman and a man, and slightly overweight
an old man wearing a beret, a gray shirt, and black pants, facing right
